# Elo Rating for Pokemon TCG Majors since 2024

The goal of this project is to assign every player at major Pokemon TCG tournaments an Elo rating, using a similar algorithm to FIDE.

The final output is a dataframe with each player's name and country in form of 'Joe Shmoe [US]' and their current Elo. It might be useful to separate first name, last name, and country later, but likely I can just merge this table with the roster in tableau.

I'd like to use this to filter games to give a matchup chart of only players above a certain Elo.

In [123]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import copy
import numpy as np

In [124]:
# Set the tournament name with the year and city of the regional

tournament = 'pokemon-bologna-2025'

previous_tournament = 'pokemon-portland-2025'

homepage = 'https://rk9.gg/event/' + tournament

elo_ratings = pd.read_csv('Elo_Ratings/updated_for_' + previous_tournament +'.csv', index_col=[0])
#elo_ratings = pd.DataFrame({}, columns = ["Name","Rating"])  #This creates an empty elo sheet

This fuction we have used before. It fetches the urls for the roster and the pairings from the rk9 homepage.

In [125]:
def get_rk9_urls(homepage):
    soup = BeautifulSoup(requests.get(homepage).text)

    tcg = soup.find("div", class_ = 'card h-100 mt-3 p-2 shadow bg-blue-050') # Locate tcg box

    # Find url extensions for roster and pairings
    roster_code = tcg.find('a', {'href': re.compile('/roster*')})['href']  
    pairings_code = tcg.find('a', {'href': re.compile('/pairings*')})['href']

    roster_url = 'https://rk9.gg' + roster_code
    pairings_url = 'https://rk9.gg' + pairings_code
    
    return [roster_url,pairings_url]

We'll use a slightly simplified version of the FIDE agorithm for calculating elo. FIDE adjusts the value of k based on historical number of games played, which I am currently not recording, as well as adjusting it for high Elo players so their ranking is not too volatile. I may implement these changes in the future.

In [126]:
def update_elo(p1_rating, p2_rating, p1_result): # For p1_result, p1 wins : 1, tie : 0.5 , p1 loses : 0

    k = 32 #Might want to make this dynamic
    
    if type(p1_result) == float: # this deals with an issue cause by double game loss penalties

        prob_p1_wins = 1/(1 + 10**((p2_rating - p1_rating)/400))
        #prob_p2_wins = 1 - prob_p1_wins

        p1_rating_gain = k*(p1_result - prob_p1_wins)

        p1_rating += p1_rating_gain
        p2_rating -= p1_rating_gain

        return [p1_rating, p2_rating]  #Should ties not count?
    
    else:
        return [p1_rating, p2_rating]

In [127]:
def run_event(homepage, elo_ratings):
    
    # Get the pairings from rk9
    
    pairings_url = get_rk9_urls(homepage)[1]
    
    total_soup = BeautifulSoup(requests.get(pairings_url).text)
    total_rounds = int(total_soup.find_all('a', id = re.compile('P2R*'))[-2].text[1:])
    
    #total_rounds = 2
    
    for round_number in range(1, total_rounds+1):
        print('running round ' + str(round_number))
        pairing_soup = BeautifulSoup(requests.get(pairings_url, {'pod' : '2', 'rnd' : str(round_number)}).text)
    
        # Set up games as a list of the games played, and set a list of player 1 and player 2 for all games.

        games = pairing_soup.find_all('div', class_ = "row row-cols-3 match no-gutter complete")
        P1_names = [game.find('span', class_ = 'name').text for game in games]
        P2_names = [game.find_all('span', class_ = 'name')[-1].text for game in games]

        # Grab the match results, and deal with dropped players

        P1_result = [game.find('div', class_ = re.compile("col-5 text-center player*"))['class'][-1] for game in games]
        for i in range(len(P1_result)):
            game = games[i]
            if P1_result[i] == 'dropped':
                P1_result[i] = game.find('div', class_ = re.compile("col-5 text-center player*"))['class'][4]
                
            if P1_result[i] == 'dropped':
                P1_result[i] = 'double game loss'

        # Generate a DataFrame for the round

        round_dict = {
            'Player 1' : P1_names,
            'Player 2' : P2_names,
            'Result' : P1_result
            }

        round_df = pd.DataFrame(round_dict)

        # Remove players that didn't play

        mask = (round_df['Player 1'] == round_df['Player 2'])
        round_df = round_df[~mask]

        # Convert 'winner', 'loser', or 'tie' to 1, 0, or 0.5

        round_df.loc[round_df['Result'] == "loser",'Result'] = 0.0
        round_df.loc[round_df['Result'] == "winner",'Result'] = 1.0
        round_df.loc[round_df['Result'] == 'tie', 'Result'] = 0.5

        # Convert the Dataframe back to 3 lists (Probably didn't need to bother with the df...)

        p1_list = round_df['Player 1'].to_list()
        p2_list = round_df['Player 2'].to_list()
        result_list = round_df['Result'].to_list()

        new_players = []  # To store unranked players

        

        for match in range(len(round_df)):  # Iterate through the games in the round
            p1_new = False    # Assume players aren't new to start
            p2_new = False
            
            p1 = p1_list[match]
            p2 = p2_list[match]      # Set players and result
            
            #p1 = re.sub(r'^\*\S+\s*', '', p1)  # This fixes vip and fixed seating issues
            #p2 = re.sub(r'^\*\S+\s*', '', p2)  # this regex removes the first word if the string starts with *
            
            #p1 = re.sub(r'^\*\S+\s+\S+\s*', '', p1)  
            #p2 = re.sub(r'^\*\S+\s+\S+\s*', '', p2)  # this regex removes the first 2 words if the string starts with *
            
            #p1 = re.sub(r'^\[[^]]*\]\s*', '', p1)  #They changed how they represent static and vip seating..
            #p2 = re.sub(r'^\[[^]]*\]\s*', '', p2)  #this regex removes the first [...] if its at the start of the string
            
            #p1 = re.sub(r'>[^>]*>\s?', '', p1)
            #p2 = re.sub(r'>[^>]*>\s?', '', p2) #this regex removes anything enlosed in > >
            
            #p1 = re.sub(r'^>\S+\s*', '', p1)
            #p2 = re.sub(r'^>\S+\s*', '', p2) # removes the first word if the string starts with >
            
            #p1 = re.sub(r'^#\S+\s*', '', p1)
            #p2 = re.sub(r'^#\S+\s*', '', p2) # removes the first word if the string starts with #
            
            #p1 = re.sub(r'^>\S+\s+\S+\s*', '', p1)  
            #p2 = re.sub(r'^>\S+\s+\S+\s*', '', p2)  # removes the first 2 words if the string starts with >
            
            #p1 = re.sub(r'\d+', '', p1)
            #p2 = re.sub(r'\d+', '', p2) #removes numbers
            
            #p1 = re.sub(r'^>\s*\S+\s*', '', p1)
            #p2 = re.sub(r'^>\s*\S+\s*', '', p2) #use this if the static seating looks like > 315 first last
            
            #p1 = re.sub(r'\bSTATIC SEATING\b|\([^)]*\)', '', p1).strip()
            #p2 = re.sub(r'\bSTATIC SEATING\b|\([^)]*\)', '', p2).strip() # for 'STATIC SEATING (30)'
            
            result = result_list[match]

            if p1 in elo_ratings.Name.values:  # This checks if the player is ranked yet

                                        # This checks if the player is a duplicate. We should only get a ValueError
                                        # if this is the case, but theres prob a cleaner way to handle this
                try:
                    old_p1_rating = elo_ratings.loc[elo_ratings['Name'] == p1, 'Rating'].item()
                except ValueError:
                    old_p1_rating = 'Duplicate'

            else:                 # Deals with new players
                p1_new = True
                old_p1_rating = 1000.0
                print(p1+' added to rankings')


            # All the same as for p1

            if p2 in elo_ratings.Name.values:
                try:
                    old_p2_rating = elo_ratings.loc[elo_ratings['Name'] == p2, 'Rating'].item()
                except ValueError:
                    old_p2_rating = 'Duplicate'
            else:
                p2_new = True
                old_p2_rating = 1000.0
                print(p2+' added to rankings')


            # For now we just can't rank duplicates
            
            if old_p1_rating == "Duplicate":
                new_p1_rating = "Duplicate"
                
            if old_p2_rating == "Duplicate":
                new_p2_rating = "Duplicate"


            if old_p1_rating != "Duplicate" and old_p2_rating != "Duplicate":

                new_ratings = update_elo(old_p1_rating, old_p2_rating, result) # Calc the new ratings

                new_p1_rating = new_ratings[0]
                new_p2_rating = new_ratings[1]

                # Assign ratings differently if they are new

                if p1_new:
                    new_players.append([p1, new_p1_rating])

                else:
                    elo_ratings.loc[elo_ratings['Name'] == p1, 'Rating'] = new_p1_rating
                if p2_new:
                    new_players.append([p2, new_p2_rating])
                else:
                    elo_ratings.loc[elo_ratings['Name'] == p2, 'Rating'] = new_p2_rating
                    
            
        # Turn the list of new players and their elos into a df, and concat with the full list

        new_players_df = pd.DataFrame(new_players,columns = ['Name', 'Rating'])
        
        #pattern = re.compile(r'^\*\S+\s*') # This fixes vip and fixed seating issues
        #new_players_df['Name'] = new_players_df['Name'].str.replace(pattern, '', regex=True)
        
        elo_ratings = pd.concat([elo_ratings, new_players_df],axis = 0,ignore_index = True)
        

    return elo_ratings

In [128]:
final_ratings = run_event(homepage, elo_ratings)
#final_ratings.to_csv('Elo_Ratings/updated_for_' + tournament +'.csv')

running round 1
Giacomo Treppiedi [IT] added to rankings
Manuel Giuseppe La Iacona [IT] added to rankings
Elisa Piccolantonio [IT] added to rankings
Graziano Piccone [IT] added to rankings
Nathan VILLA [FR] added to rankings
andrea minopoli [IT] added to rankings
Nicola Bono [IT] added to rankings
Nicola Poggiana [IT] added to rankings
Jan Breznar [SI] added to rankings
Luca Bandini [IT] added to rankings
Gabriele Frantozzi [IT] added to rankings
Alessio De Nicola [IT] added to rankings
Serena Anna Iaconis [IT] added to rankings
Alessandro Pula [IT] added to rankings
Marco Pieroni [IT] added to rankings
Daniele Iofalo [IT] added to rankings
Jonny Maculan [IT] added to rankings
Sebastiano Cantore [IT] added to rankings
Alessandro Di Patrizio [IT] added to rankings
Riccardo Rossetti [IT] added to rankings
Roberta Buia [IT] added to rankings
Samuele Ciardelli [IT] added to rankings
Tommaso Falchi [IT] added to rankings
Andrea Gilioli [IT] added to rankings
Domenico Paladino [IT] added to 

Erik Schetters [DE] added to rankings
Paolo Varbaro [IT] added to rankings
Gaetano Granata [IT] added to rankings
Justin Robert Welke [DE] added to rankings
Vittorio Ruffinengo [IT] added to rankings
Lorenzo Contu [IT] added to rankings
Mirco Quatrale [IT] added to rankings
Marica Rossi [IT] added to rankings
Valerio Carista [IT] added to rankings
Filippo Succi [IT] added to rankings
Lapo Lepri [IT] added to rankings
leonardo Giannotta [IT] added to rankings
Matteo Bonginelli [IT] added to rankings
Davide Berrino [IT] added to rankings
Alessandro Laudini [IT] added to rankings
Marwan ATTOBI [FR] added to rankings
Tommaso Foresi [IT] added to rankings
Francesco Festa [IT] added to rankings
Michele Fabiano [IT] added to rankings
Leonardo Benassi [IT] added to rankings
Lorenzo Tolotti [IT] added to rankings
Manuel Piccardi [IT] added to rankings
Davide Abbondandolo [IT] added to rankings
Rob Bartlett [UK] added to rankings
Emanuele Perrone Guerzoni [IT] added to rankings
Pasquale Porrari 

Francesco Muroni [IT] added to rankings
Maurizio Gozzi [IT] added to rankings
Matteo Caldaroni [IT] added to rankings
Fin Peters [UK] added to rankings
Giacomo Burzi [IT] added to rankings
Alessio Marchetti [IT] added to rankings
Chiara Martinis [IT] added to rankings
Diego Ebau [IT] added to rankings
Aaron Dall'Era [IT] added to rankings
Santiago de Saussure [CH] added to rankings
Daniele Ricci [IT] added to rankings
Gianluca Sarcina [IT] added to rankings
Gabriele Pula [IT] added to rankings
Tommaso Napolitano [IT] added to rankings
Diego Battistini [IT] added to rankings
Alessandro Cagliano [IT] added to rankings
Francesco Cimmino [IT] added to rankings
Carmine Scaglione [IT] added to rankings
Paolo Antinori [IT] added to rankings
Gabriele Garcina [IT] added to rankings
Emanuele Seghetti [IT] added to rankings
manuel domenici [IT] added to rankings
Gabriele Borghetti [IT] added to rankings
alex birzu [IT] added to rankings
Pietro Piazze [IT] added to rankings
Francesco Nardi [IT] ad

In [129]:
final_ratings[final_ratings['Rating'] >= 1350]

,Name,Rating
73,Abaan Ahmed [US],1423.396623
157,Tord Reklev [NO],1355.197925
648,Sebastian Lashmet [US],1391.909109
759,Isaiah Bradner [US],1392.071009
798,Caleb Rogerson [US],1450.537702
833,Andrew Hedrick [US],1386.503092
896,Michael Davidson [US],1432.432443
915,Alex Schemanske [US],1391.206703
925,Lucas Xing [CA],1415.672654
1075,Grant Shen [US],1366.346494


In [131]:
final_ratings = final_ratings.drop_duplicates(subset = 'Name', keep = False)
final_ratings

,Name,Rating
0,Aaron Walker [US],1201.293823
1,Jimmy Wellington [US],936.000000
2,Cassidy Gibson [US],996.628463
3,Kenneth Mauder [US],1236.549829
4,Jacob Henry [US],892.906995
...,...,...
33670,Filippo Ferrari [IT],981.275649
33671,Luca Capodiferro,986.661861
33672,Michael Monteleone,995.788140
33673,Daniele Bertoncini,1045.003199


In [137]:
final_ratings.to_csv('Elo_Ratings/updated_for_' + tournament +'.csv')